In [6]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [22]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import random

In [42]:
P = 17;
G = 6;

d = random.randint(1, 1000)

public_key_d = (G**d)%P

with open('shared_file', 'rb') as f:
    public_key_c = f.read()
    public_key_c = int.from_bytes(public_key_c, byteorder='big', signed=False)
    print(public_key_c)

sD = (public_key_c**d)%P


5
